## Hometask 

1) Classify the signs (fingers)  dataset 

2) Try to change the number of hidden layer 

3) Change the activation to tanh or sigmoid and see what happens

4) Change the dropout ratio and check the performance

In [12]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

import torch

In [3]:
def load_dataset():
    fn = 'supplementary_data/train_signs.h5'
    train_dataset = h5py.File(fn, "r")
    X_train = np.array(train_dataset["train_set_x"][:]) # your train set features
    Y_train = np.array(train_dataset["train_set_y"][:]) # your train set labels

    fn = 'supplementary_data/test_signs.h5'
    test_dataset =  h5py.File(fn, "r")
    X_test = np.array(test_dataset["test_set_x"][:]) # your test set features
    Y_test = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    Y_train = Y_train.reshape((1, Y_train.shape[0]))
    Y_test = Y_test.reshape((1, Y_test.shape[0]))
    
    return X_train, Y_train, X_test, Y_test, classes

train_data, train_labels, test_data, test_labels, classes = load_dataset()

print ('train_data.shape=', train_data.shape)
print ('train_labels.shape=',  train_labels.shape)
print ('test_data.shape=', test_data.shape)
print ('test_labels.shape=', test_labels.shape)

train_data.shape= (1080, 64, 64, 3)
train_labels.shape= (1, 1080)
test_data.shape= (120, 64, 64, 3)
test_labels.shape= (1, 120)


In [24]:
#preprocess the data
X_train = train_data.astype('float32')
X_test = test_data.astype('float32')

X_train /= 255.0
X_test /= 255.0

# Change from matrix to array of dimension 64x64 to array of dimension 12288
dimData = np.prod(X_train.shape[1:])
X_train = X_train.reshape(X_train.shape[0], dimData) # think could be just -1 
X_test = X_test.reshape(X_test.shape[0], dimData)

Y_train_one_hot = to_categorical(train_labels.flatten())
Y_test_one_hot = to_categorical(test_labels.flatten())

print(dimData, X_train.shape, X_test.shape)
print('Number of classes: ', len(classes))
print('Original label [0] : ', train_labels[0])
print('After conversion to categorical ( one-hot ) : ', Y_train_one_hot[0])

12288 (1080, 12288) (120, 12288)
Number of classes:  6
Original label [0] :  [5 0 2 ... 2 4 5]
After conversion to categorical ( one-hot ) :  [0. 0. 0. 0. 0. 1.]


In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [29]:
import keras

class MyModel(keras.Model):
    def __init__(self, activation='relu', dropout_rate=0.3, hidden_units=[512, 256], num_classes=6):
        super().__init__()
        self.hidden_layers = []
        for units in hidden_units:
            self.hidden_layers.append(Dense(units, activation=activation))
            self.hidden_layers.append(Dropout(dropout_rate))
        self.output_layer = Dense(num_classes, activation='softmax')
    def call(self, inputs):
        x = inputs
        for layer in self.hidden_layers:
            x = layer(x)
        return self.output_layer(x)

In [33]:
dropouts = [0.1, 0.3, 0.5, 0.7]
activations = ['relu', 'sigmoid', 'tanh']
hidden_units_list = [[512, 256], [256, 128], [128, 64], [512, 256, 128], [128, 256, 128]]

In [36]:
import tensorflow as tf
print(f"Available devices: {tf.config.list_physical_devices()}")
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [ ]:
import itertools

for dropout, activation, hidden_units in itertools.product(dropouts, activations, hidden_units_list):
    with tf.device(device):
        print(f"Training model with dropout={dropout}, activation={activation}, hidden_units={hidden_units}")
        
        model = MyModel(activation=activation, dropout_rate=dropout, hidden_units=hidden_units, num_classes=6)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
        history = model.fit(
            X_train, Y_train_one_hot,
            validation_data=(X_test, Y_test_one_hot),
            epochs=20,
            batch_size=32,
            verbose=1
        )

        val_acc = history.history['val_accuracy'][-1]
        print(f"Validation accuracy: {val_acc:.4f}\n")

Training model with dropout=0.1, activation=relu, hidden_units=[512, 256]
Epoch 1/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.1625 - loss: 1.7940 - val_accuracy: 0.1667 - val_loss: 1.7845
Epoch 2/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.1959 - loss: 1.7822 - val_accuracy: 0.1667 - val_loss: 1.7650
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.2180 - loss: 1.7673 - val_accuracy: 0.3583 - val_loss: 1.7191
Epoch 4/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.2996 - loss: 1.7161 - val_accuracy: 0.3583 - val_loss: 1.6498
Epoch 5/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.3065 - loss: 1.6469 - val_accuracy: 0.3000 - val_loss: 1.5962
Epoch 6/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.3567 - loss: 1.5807 - val_accuracy: 0.2917 - val_loss: 1.5109
Epoch 7/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.4149 - loss: 1.4914 - val_accuracy: 0.4667 - val_loss: 1.3516
Epoch 8/20
34/34 ━━━━━━━━━━━━━━━━━━━━

Activation Functions:
`tanh` consistently outperformed others, especially with higher dropout rates — delivering stable and strong accuracy. `ReLU` performed decently but didn't stand out, while `sigmoid` was generally disappointing, rarely exceeding 45%.

Network Depth:
More layers didn’t necessarily lead to better performance. In many cases, two-layer networks outperformed three-layer ones. For example, with `tanh` and `dropout=0.1`, the [128, 64] model achieved 70%, while a deeper [128, 256, 128] setup only reached 65.83%. It seems the task isn't complex enough to require deep architectures.

Layer Size:
Compact models like [128, 64] often performed as well as larger ones like [512, 256], suggesting the dataset doesn’t benefit significantly from high-capacity networks.

The best config is - dropout=0.5, activation=tanh, hidden_units=[512, 256] with accuracy 0.8167